In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

gpt_model = AutoModelForCausalLM.from_pretrained("gpt2-medium")
tokenizer = AutoTokenizer.from_pretrained("gpt2-medium", device_map="auto")

In [602]:
from latent import *
model = LatentLM("gpt2-medium", model=gpt_model, tokenizer=tokenizer)

In [555]:
r = model(["How", "What are you up to"]).layer(-1)
r

<gpt2-medium.layer(-1).LatentTensor[2, 6] <-> 'How[TOK]'
                                              'What are you up to[TOK]'

In [564]:
print(r.complete())

<gpt2-medium.LatentTensor[25, 2, 7] <-> 'How did[TOK]'
                                        'What are you up to?[TOK]'



In [567]:
r.complete()

<gpt2-medium.LatentTensor[25, 2, 7] <-> 'How did[TOK]'
                                        'What are you up to?[TOK]'

In [590]:
r.distribution()

tensor([[ -5.6751,  -8.6013, -13.2165,  ..., -18.4128, -12.4559, -10.6140],
        [ -5.5568, -10.8951, -11.8829,  ..., -19.0959, -19.4955,  -9.3850]],
       grad_fn=<LogSoftmaxBackward0>)

In [592]:
r.layer(-1)

<gpt2-medium.layer(-1).LatentTensor[2, 6] <-> 'How[TOK]'
                                              'What are you up to[TOK]'

In [593]:
from latent import *

class Adapter(LatentModule):
    def __init__(self):
        super().__init__()
        
        self.lin1 = torch.nn.Linear(1024, 1024)
        # self.lin2 = torch.nn.Linear(32, 4096)

    def forward(self, x):
        x = self.lin1(x)
        return x
    
adapter = Adapter().to(model.device)

def color_match(color: str):
    soft_color = model(f"The boat is {color}.", name='soft_color')[-2]
    soft_color = adapter(soft_color)
    x = model(soft_color, "The color of the boat is", name='soft_color')
    
    y = model(f"The boat is {color}. The color of the boat is", name='hard_color')
    
    return x[-1] == y[-1]
color_match("red")

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:24                                                                                   │
│                                                                                                  │
│   21 │   y = model(f"The boat is {color}. The color of the boat is", name='hard_color')          │
│   22 │                                                                                           │
│   23 │   return x[-1] == y[-1]                                                                   │
│ ❱ 24 color_match("red")                                                                          │
│   25                                                                                             │
│                                                                                                  │
│ in color_match:18                                                                                │
│                                                                                                  │
│   15                                                                                             │
│   16 def color_match(color: str):                                                                │
│   17 │   soft_color = model(f"The boat is {color}.", name='soft_color')[-2]                      │
│ ❱ 18 │   soft_color = adapter(soft_color)                                                        │
│   19 │   x = model(soft_color, "The color of the boat is", name='soft_color')                    │
│   20 │                                                                                           │
│   21 │   y = model(f"The boat is {color}. The color of the boat is", name='hard_color')          │
│                                                                                                  │
│ /home/luca/repos/lat/latent.py:619 in __call__                                                   │
│                                                                                                  │
│   616 │   │   super().__init__()                                                                 │
│   617 │                                                                                          │
│   618 │   def __call__(self, latent: LatentTensor, *args, **kwargs) -> LatentTensor:             │
│ ❱ 619 │   │   x = self.forward(latent.activations, *args, **kwargs)                              │
│   620 │   │   name = f"{self.name}({latent.name})"                                               │
│   621 │   │   return LatentTensor(x, latent.prompts, latent.model, latent.input_ids, latent.of   │
│   622                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'LatentTensor' object has no attribute 'activations'

In [608]:
color = "pink"
model([f"The boat is {color}. The color of the boat is"], name='hard_color').complete()

<gpt2-medium.LatentTensor[25, 1, 13] <-> 'The boat is pink. The color of the boat is pink[TOK]'

In [610]:
color = "pink"
soft_color = model(f"The boat is {color}.", name='soft_color')[-2]
soft_color = adapter(soft_color)
x = model(soft_color, "The color of the boat is", name='soft_color').complete()
x

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:3                                                                                    │
│                                                                                                  │
│   1 color = "pink"                                                                               │
│   2 soft_color = model(f"The boat is {color}.", name='soft_color')[-2]                           │
│ ❱ 3 soft_color = adapter(soft_color)                                                             │
│   4 x = model(soft_color, "The color of the boat is", name='soft_color').complete()              │
│   5 x                                                                                            │
│   6                                                                                              │
│                                                                                                  │
│ /home/luca/repos/lat/latent.py:616 in __call__                                                   │
│                                                                                                  │
│   613 │   │   super().__init__()                                                                 │
│   614 │                                                                                          │
│   615 │   def __call__(self, latent: LatentTensor, *args, **kwargs) -> LatentTensor:             │
│ ❱ 616 │   │   x = self.forward(latent.hidden_states, *args, **kwargs)                            │
│   617 │   │   name = f"{self.name}({latent.name})"                                               │
│   618 │   │   return LatentTensor(x, latent.prompts, latent.model, latent.input_ids, latent.of   │
│   619                                                                                            │
│                                                                                                  │
│ in forward:11                                                                                    │
│                                                                                                  │
│    8 │   │   # self.lin2 = torch.nn.Linear(32, 4096)                                             │
│    9 │                                                                                           │
│   10 │   def forward(self, x):                                                                   │
│ ❱ 11 │   │   x = self.lin1(x)                                                                    │
│   12 │   │   return x                                                                            │
│   13                                                                                             │
│   14 adapter = Adapter().to(model.device)                                                        │
│                                                                                                  │
│ /home/luca/miniconda3/envs/lmql/lib/python3.10/site-packages/torch/nn/modules/module.py:1501 in  │
│ _call_impl                                                                                       │
│                                                                                                  │
│   1498 │   │   if not (self._backward_hooks or self._backward_pre_hooks or self._forward_hooks   │
│   1499 │   │   │   │   or _global_backward_pre_hooks or _global_backward_hooks                   │
│   1500 │   │   │   │   or _global_forward_hooks or _global_forward_pre_hooks):                   │
│ ❱ 1501 │   │   │   return forward_call(*args, **kwargs)                                          │
│   1502 │   │   # Do not call functions when jit is used                                          │
│   1503 │   │   full_backward_hooks, non_full_backward_hooks = [], []                             │
│   1504 │   │   backward_pre_hooks = []                     

In [8]:
colors = []
with open("colors.txt") as f:
    for color in f:
        colors.append(color.strip())
print(colors)

['alizarin', 'amaranth', 'amber', 'amethyst', 'apricot', 'aqua', 'aquamarine', 'asparagus', 'auburn', 'azure', 'beige', 'bistre', 'black', 'blue', 'blue-green', 'blue-violet', 'bondi-blue', 'brass', 'bronze', 'brown', 'buff', 'burgundy', 'camouflage-green', 'caput-mortuum', 'cardinal', 'carmine', 'carrot-orange', 'celadon', 'cerise', 'cerulean', 'champagne', 'charcoal', 'chartreuse', 'cherry-blossom-pink', 'chestnut', 'chocolate', 'cinnabar', 'cinnamon', 'cobalt', 'copper', 'coral', 'corn', 'cornflower', 'cream', 'crimson', 'cyan', 'dandelion', 'denim', 'ecru', 'emerald', 'eggplant', 'falu-red', 'fern-green', 'firebrick', 'flax', 'forest-green', 'french-rose', 'fuchsia', 'gamboge', 'gold', 'goldenrod', 'green', 'grey', 'han-purple', 'harlequin', 'heliotrope', 'hollywood-cerise', 'indigo', 'ivory', 'jade', 'kelly-green', 'khaki', 'lavender', 'lawn-green', 'lemon', 'lemon-chiffon', 'lilac', 'lime', 'lime-green', 'linen', 'magenta', 'magnolia', 'malachite', 'maroon', 'mauve', 'midnight-bl

In [ ]:
train, test = colors[:100], colors[100:]
adapter.fit(color_match, train, epochs=100, lr=1e-4, loss_fct="mse", test=test)

Epoch 1/100, loss=0.6019:   1%|█                                                                                                        | 100/10000 [01:19<2:11:38,  1.25it/s]

Loss: 0.6019 Train accuracy: 15.00% Test accuracy: 24.24%


Epoch 2/100, loss=0.4452:   2%|██                                                                                                       | 200/10000 [03:09<2:09:57,  1.26it/s]

Loss: 0.4452 Train accuracy: 28.00% Test accuracy: 37.88%


Epoch 3/100, loss=0.4221:   3%|███▏                                                                                                     | 300/10000 [04:59<2:08:37,  1.26it/s]

...

Epoch 10/100, loss=0.1810:  10%|██████████▎                                                                                            | 1000/10000 [17:52<1:59:26,  1.26it/s]

Loss: 0.1810 Train accuracy: 29.00% Test accuracy: 37.88%


Epoch 11/100, loss=0.1881:  11%|███████████▎                                                                                           | 1100/10000 [19:42<1:58:09,  1.26it/s]

Loss: 0.1881 Train accuracy: 30.00% Test accuracy: 37.88%


Epoch 12/100, loss=0.1428:  12%|████████████▎                                                                                          | 1200/10000 [21:32<1:57:37,  1.25it/s]